In [1]:
## download project & convert TF1 to TF2.
!git clone https://github.com/JiahuiYu/generative_inpainting
!tf_upgrade_v2 --intree generative_inpainting --outtree generative_inpainting_tf2

## download support module & convert TF1 to TF2.
!git clone https://github.com/JiahuiYu/neuralgym
!tf_upgrade_v2 --intree neuralgym/neuralgym/ --outtree generative_inpainting_tf2/neuralgym

Cloning into 'generative_inpainting'...
remote: Enumerating objects: 193, done.
remote: Total 193 (delta 0), reused 0 (delta 0), pack-reused 193
Receiving objects: 100% (193/193), 13.77 MiB | 29.24 MiB/s, done.
Resolving deltas: 100% (63/63), done.
2023-05-03 14:09:42.018710: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 14:09:43.990353: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO line 44:18: Renamed 'tf.ConfigProto' to 'tf.compat.v1.ConfigProto'
INFO line 46:9: Renamed 'tf.Session' to 'tf.compat.v1.Session'
INFO line 53:20: Renamed 'tf.get_collection' to 'tf.compat.v1.get_collection'
INFO line 53:38: Renamed 'tf.GraphKeys' to 'tf.compat.v1.GraphKeys'
INFO line 58:24: Renamed 'tf.contr

In [2]:
## apply patch

%%bash
cd /content/generative_inpainting_tf2/
cat << EOS | git apply
diff --git a/inpaint_model.py b/inpaint_model.py
index 6e8ca5d..93ccf2e 100644
--- a/inpaint_model.py
+++ b/inpaint_model.py
@@ -4,7 +4,7 @@ import logging
 import cv2
 import neuralgym as ng
 import tensorflow as tf
-from tensorflow.contrib.framework.python.ops import arg_scope
+from tf_slim import arg_scope
 
 from neuralgym.models import Model
 from neuralgym.ops.summary_ops import scalar_summary, images_summary
diff --git a/inpaint_ops.py b/inpaint_ops.py
index 740ac46..66ef60b 100644
--- a/inpaint_ops.py
+++ b/inpaint_ops.py
@@ -4,7 +4,7 @@ import math
 import cv2
 import numpy as np
 import tensorflow as tf
-from tensorflow.contrib.framework.python.ops import add_arg_scope
+from tf_slim import add_arg_scope
 from PIL import Image, ImageDraw
 
 from neuralgym.ops.layers import resize
diff --git a/neuralgym/ops/layers.py b/neuralgym/ops/layers.py
index d4fabbc..ed35226 100644
--- a/neuralgym/ops/layers.py
+++ b/neuralgym/ops/layers.py
@@ -494,4 +494,4 @@ def flatten(x, name='flatten'):
 
     """
     with tf.compat.v1.variable_scope(name):
-        return tf.contrib.layers.flatten(x)
+        return tf.compat.v1.layers.flatten(x)
EOS

In [3]:
!gdown -O generative_inpainting_tf2/model --folder https://drive.google.com/drive/folders/1y7Irxm3HSHGvp546hZdAZwuNmhLUVcjO?usp=sharing 

Retrieving folder list
Processing file 1dyPD2hx0JTmMuHYa32j-pu--MXqgLgMy checkpoint
Processing file 1z9dbEAzr5lmlCewixevFMTVBmNuSNAgK snap-0.data-00000-of-00001
Processing file 1ExY4hlx0DjVElqJlki57la3Qxu40uhgd snap-0.index
Processing file 1C7kPxqrpNpQF7B2UAKd_GvhUMk0prRdV snap-0.meta
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1dyPD2hx0JTmMuHYa32j-pu--MXqgLgMy
To: /content/generative_inpainting_tf2/model/checkpoint
100% 32.0/32.0 [00:00<00:00, 173kB/s]
Downloading...
From: https://drive.google.com/uc?id=1z9dbEAzr5lmlCewixevFMTVBmNuSNAgK
To: /content/generative_inpainting_tf2/model/snap-0.data-00000-of-00001
100% 120M/120M [00:00<00:00, 228MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ExY4hlx0DjVElqJlki57la3Qxu40uhgd
To: /content/generative_inpainting_tf2/model/snap-0.index
100% 11.5k/11.5k [00:00<00:00, 40.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=

In [4]:
## test run

%cd /content/generative_inpainting_tf2/
!python test.py --image examples/places2/case1_input.png --mask examples/places2/case1_mask.png --output case1_output.png --checkpoint_dir model

/content/generative_inpainting_tf2
2023-05-03 14:10:07.735915: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 14:10:08.864523: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
---------------------------------- APP CONFIG ----------------------------------
num_gpus_per_job: 1
num_cpus_per_job: 4
num_hosts_per_job: 1
memory_per_job: 32
gpu_type: nvidia-tesla-p100
name: places2_gated_conv_v100
model_restore: 
dataset: celebahq
random_crop: False
val: False
log_dir: logs/full_model_celeba_hq_256
gan: sngan
gan_loss_alpha: 1
gan_with_mask: True
discounted_mask: True
random_seed: False
padding: SAME
train_spe: 4000
max_iters: 100000000
viz_max_out: 10
val_psteps: 2000
data_flist: 
  celebahq: ['data/c

In [5]:
## convert the model to tflite

import numpy as np
import tensorflow as tf
import neuralgym as ng
from inpaint_model import InpaintCAModel

CHECKPOINT = './model'
TFLITE     = 'deepfillv2.tflite'

FLAGS = ng.Config('inpaint.yml')
model = InpaintCAModel()

def saturate_cast_u8(x):
  x = tf.math.maximum(x, tf.constant([0.0], dtype=tf.float32))
  x = tf.math.minimum(x, tf.constant([255.0], dtype=tf.float32))
  x = tf.cast(x, dtype=tf.uint8)
  return x

sess_config = tf.compat.v1.ConfigProto()
sess_config.gpu_options.allow_growth = True
with tf.compat.v1.Session(config=sess_config) as sess:
  input_image = tf.cast(tf.random.uniform([1,512,1360,3], maxval=255, dtype=tf.int32), tf.float32)
  output = model.build_server_graph(FLAGS, input_image)
  output = (output + 1.) * 127.5
  output = tf.reverse(output, [-1])
  output = saturate_cast_u8(output)
  # load pretrained model
  vars_list = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES)
  assign_ops = []
  for var in vars_list:
    vname = var.name
    var_value = tf.train.load_variable(CHECKPOINT, vname)
    assign_ops.append(tf.compat.v1.assign(var, var_value))
  sess.run(assign_ops)
  print('Model loaded.')

  # save the model as saved_model
  tf.compat.v1.saved_model.simple_save(
    sess,
    './saved_model',
    inputs={'input': input_image},
    outputs={'output': output})

# convert to tflite
converter = tf.lite.TFLiteConverter.from_saved_model('./saved_model')
converter.allow_custom_ops = True
tflite_model = converter.convert()

open(TFLITE, "wb").write(tflite_model)

---------------------------------- APP CONFIG ----------------------------------
num_gpus_per_job: 1
num_cpus_per_job: 4
num_hosts_per_job: 1
memory_per_job: 32
gpu_type: nvidia-tesla-p100
name: places2_gated_conv_v100
model_restore: 
dataset: celebahq
random_crop: False
val: False
log_dir: logs/full_model_celeba_hq_256
gan: sngan
gan_loss_alpha: 1
gan_with_mask: True
discounted_mask: True
random_seed: False
padding: SAME
train_spe: 4000
max_iters: 100000000
viz_max_out: 10
val_psteps: 2000
data_flist: 
  celebahq: ['data/celeba_hq/train_shuffled.flist', 'data/celeba_hq/validation_static_view.flist']
  celeba: ['data/celeba/train_shuffled.flist', 'data/celeba/validation_static_view.flist']
  places2: ['data/places2/train_shuffled.flist', 'data/places2/validation_static_view.flist']
  imagenet: ['data/imagenet/train_shuffled.flist', 'data/imagenet/validation_static_view.flist']
static_view_size: 30
img_shapes: [256, 256, 3]
height: 128
width: 128
max_delta_height: 32
max_delta_width: 32

/content/generative_inpainting_tf2/inpaint_ops.py:46: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  x = tf.compat.v1.layers.conv2d(
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/mi

Model loaded.


17685392

In [ ]:
## download the converted model - include saved_model & tflite

!zip -r tfl_deepfillv2.zip saved_model deepfillv2.tflite
from google.colab import files
files.download('/content/generative_inpainting_tf2/tfl_deepfillv2.zip')

In [ ]:
## download project

!cd ..; zip -r generative_inpainting_tf2.zip ./generative_inpainting_tf2
files.download('/content/generative_inpainting_tf2.zip')